In [1]:
# Cell 1 - Fix the function definition and variable scope
import os, time, jwt, requests
from dotenv import load_dotenv; load_dotenv()

APP_ID = os.environ["GITHUB_APP_ID"]                 
INSTALLATION_ID = os.environ["GITHUB_INSTALLATION_ID"]
with open(os.environ["GITHUB_APP_PRIVATE_KEY_PEM"], "r") as f:
    PRIVATE_KEY = f.read()

def make_app_jwt(app_id: str, pem: str) -> str:  # Fixed: added 'def make'
    now = int(time.time())
    payload = {
        "iat": now - 60,        # 60s in the past for skew
        "exp": now + 540,       # 9 minutes ahead (NOT 10)
        "iss": int(app_id)      # your GitHub App ID (numeric)
    }
    return jwt.encode(payload, pem, algorithm="RS256")

def get_installation_token(installation_id: str, app_jwt: str) -> str:
    url = f"https://api.github.com/app/installations/{installation_id}/access_tokens"
    r = requests.post(url, headers={
        "Authorization": f"Bearer {app_jwt}",
        "Accept": "application/vnd.github+json"
    })
    r.raise_for_status()
    return r.json()["token"]          # 1-hour token

# Initialize variables at module level for cross-cell access
app_jwt = None
inst_token = None

try:
    app_jwt = make_app_jwt(APP_ID, PRIVATE_KEY)
    print(f"✅ JWT created successfully")
    
    inst_token = get_installation_token(INSTALLATION_ID, app_jwt)
    print("✅ OK: got installation token (1h lifetime)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

✅ JWT created successfully
✅ OK: got installation token (1h lifetime)
